In [18]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint

In [30]:
minbal = 362894.90
maxbal = 362894.90
ax = np.array([0.2,0.05,0.04,0.04,0.17,0.05,0.12,0.15,0.08,0.1])
px = np.array([140.38,104.98,110.96,135.5,44.62,48.98,79.11,54.51,109.58,45.07])
cs = np.array([528,182,143,101,1364,367,534,986,260,816])

In [78]:
def rebalance(minbal,maxbal,ax,px,tol):
    mintrade = 1
    maxtrade = np.inf
    targetTol = tol
    x0 = np.ones(len(ax))
    bounds = [(mintrade,maxtrade)]*len(x0)
    cons = [
            dict(type='ineq',fun=lambda x: maxbal - px.dot(x)),
            dict(type='ineq',fun=lambda x: px.dot(x) - minbal),
            dict(type='ineq',fun=lambda x: tol - np.std(ax - px*x/px.dot(x)))]
    def targetoffset(x):
        w = px*x/px.dot(x)
        return np.sum(np.abs(ax-w))
    result = minimize(targetoffset, x0, constraints=cons, bounds=bounds, method='SLSQP')
    return result.success, np.round(result.x), result.message


In [85]:
ok, values, msg = rebalance(minbal,maxbal,ax,px,0.05)

In [86]:
print(ok)
print(msg)
print(values)

True
Optimization terminated successfully.
[491. 241. 142.   1. 963. 488. 820. 972. 343. 679.]


In [87]:
cs-values

array([  37.,  -59.,    1.,  100.,  401., -121., -286.,   14.,  -83.,
        137.])

In [88]:
np.round(ax - px*values/px.dot(values),2)*100

array([ 1., -2., -0.,  4.,  5., -2., -6.,  0., -2.,  2.])

In [89]:
np.std(ax - px*values/px.dot(values))

0.030415667247252455

In [90]:
np.var(ax - px*values/px.dot(values))

0.0009251128140955857